In [1]:
import pandas as pd
import numpy as np
# from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
import string
# from nltk.stem.porter import *
import nltk
from nltk.corpus import stopwords
# from nltk.corpus import wordnet
from ast import literal_eval
from tqdm import tqdm

In [ ]:
# nltk.download("stopwords")

# Content-based Filtering

## model 1: TF-IDF for movie `overview`,  `tagline`, and `keywords`

### Natual Language Processing

In [2]:
PUNCTUATION = string.punctuation
# stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")
STOPWORDS = stopwords.words("english")

In [36]:
def nlp(x):
    """
    Natural Language Processing
    
    Args:
        x (str)
    
    Returns:
        (str)
    """
    x = x.lower() # lowercase string
    x = [c for c in x if not (c in PUNCTUATION) and (not c.isdigit())] # non-punct characters
    x = "".join(x) # convert back to string
    words = x.strip().split() # tokenizes
    words = [w for w in words if w not in STOPWORDS] # remove stopwords and numbers
    return " ".join(stemmer.stem(w) for w in words)

In [24]:
def has_number(x):
    for c in x:
        if c.isdigit():
            print(x)
            break
    

In [4]:
# movie_meta = pd.read_csv("the-movies-dataset/movies_metadata.csv", dtype=str).drop_duplicates()
movie_meta = pd.read_parquet("the-movies-dataset/processed_data.parquet", engine="fastparquet").reset_index(drop=True)
# movie_meta.id = movie_meta.id.astype(int)
# movie_meta.overview.fillna("", inplace=True)
# movie_meta.tagline.fillna("", inplace=True)
movie_meta

,id,title,cast,keywords,genres,vote_average,vote_count,overview,release_date,popularity,tagline,director
0,862,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",7.7,5415,"Led by Woody, Andy's toys live happily in his ...",23950,21.946943,,[John Lasseter]
1,8844,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, new home, recluse,...","[Adventure, Fantasy, Family]",6.9,2413,When siblings Judy and Peter discover an encha...,23952,17.015539,Roll the dice and unleash the excitement!,[Joe Johnston]
2,15602,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Sophia Loren, Da...","[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]",6.5,92,A family wedding reignites the ancient feud be...,23952,11.7129,Still Yelling. Still Fighting. Still Ready for...,[Howard Deutch]
3,31357,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]",6.1,34,"Cheated on, mistreated and stepped on, the wom...",23952,3.859495,Friends are the people who let you be yourself...,[Forest Whitaker]
4,11862,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...",[Comedy],5.7,173,Just when George Banks has recovered from his ...,23942,8.387519,Just When His World Is Back To Normal... He's ...,[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...
45364,30840,Robin Hood,"[Patrick Bergin, Uma Thurman, David Morrissey,...",[],"[Drama, Action, Romance]",5.7,26,"Yet another version of the classic epic, with ...",23897,5.683753,,[John Irvin]
45365,111109,Century of Birthing,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[artist, play, pinoy]",[Drama],9.0,3,An artist struggles to finish his work while a...,24143,0.178241,,[Lav Diaz]
45366,67758,Betrayal,"[Erika Eleniak, Adam Baldwin, James Remar, Dam...",[],"[Action, Drama, Thriller]",3.8,6,"When one of her hits goes wrong, a professiona...",24044,0.903007,A deadly game of wits.,[]
45367,227506,Satan Triumphant,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[],[],0.0,0,"In a small town live two brothers, one a minis...",23014,0.003503,,[Yakov Protazanov]


In [27]:
%%time
movie_meta["description"] = movie_meta.overview + " " + movie_meta.tagline

CPU times: user 24.9 s, sys: 42 ms, total: 24.9 s
Wall time: 25 s


In [37]:
%%time
movie_meta["description_nlp"] = movie_meta["description"].apply(nlp)

CPU times: user 24.9 s, sys: 24.4 ms, total: 24.9 s
Wall time: 24.9 s


In [38]:
movie_meta["description_nlp"].apply(has_number)

0        None
1        None
2        None
3        None
4        None
         ... 
45364    None
45365    None
45366    None
45367    None
45368    None
Name: description_nlp, Length: 45369, dtype: object

In [18]:
movie_meta.iloc[1111]["description"]

'It\'s 1974. Muhammad Ali is 32 and thought by many to be past his prime. George Foreman is ten years younger and the heavyweight champion of the world. Promoter Don King wants to make a name for himself and offers both fighters five million dollars apiece to fight one another, and when they accept, King has only to come up with the money. He finds a willing backer in Mobutu Sese Suko, the dictator of Zaire, and the "Rumble in the Jungle" is set, including a musical festival featuring some of America\'s top black performers, like James Brown and B.B. King.The untold story of the Rumble in the Jungle.'

In [21]:
movie_meta.description_nlp.iloc[1111]

'muhammad ali thought mani past prime georg foreman ten year younger heavyweight champion world promot king want make name offer fighter five million dollar apiec fight one anoth accept king come money find will backer mobutu sese suko dictat zair rumbl jungl set includ music festiv featur america top black perform like jame brown bb kingth untold stori rumbl jungl'

### TF-IDF matrix

In [39]:
tfidf = TfidfVectorizer(analyzer="word", stop_words="english")

In [40]:
tfidf_matrix = tfidf.fit_transform(movie_meta.description_nlp)

In [41]:
tfidf_matrix.shape

(45369, 68611)

In [42]:
movie_meta.description_nlp[0]

'led woodi andi toy live happili room andi birthday bring buzz lightyear onto scene afraid lose place andi heart woodi plot buzz circumst separ buzz woodi owner duo eventu learn put asid differ'

In [43]:
tfidf.get_feature_names()

['aa',
 'aaa',
 'aaargh',
 'aaaron',
 'aabgamma',
 'aachan',
 'aachi',
 'aackerlund',
 'aadhavan',
 'aadland',
 'aaicha',
 'aak',
 'aakash',
 'aalavandhan',
 'aalst',
 'aaltonen',
 'aam',
 'aamir',
 'aamp',
 'aampa',
 'aampm',
 'aampr',
 'aan',
 'aanandam',
 'aang',
 'aangehouden',
 'aanmodderfakk',
 'aap',
 'aarakshan',
 'aarchi',
 'aarn',
 'aarno',
 'aaron',
 'aarp',
 'aart',
 'aarti',
 'aarya',
 'aarón',
 'aasen',
 'aashirvad',
 'aashish',
 'aasia',
 'aati',
 'aatish',
 'ab',
 'aba',
 'ababa',
 'abacco',
 'aback',
 'abacus',
 'abaddon',
 'abagnal',
 'abahachi',
 'abalaba',
 'aballay',
 'abalon',
 'abandon',
 'abandond',
 'abar',
 'abard',
 'abargil',
 'abat',
 'abatcha',
 'abati',
 'abb',
 'abba',
 'abbaji',
 'abbasi',
 'abbass',
 'abberlin',
 'abbess',
 'abbey',
 'abbi',
 'abbia',
 'abbot',
 'abbott',
 'abby',
 'abbé',
 'abc',
 'abctv',
 'abdallah',
 'abdel',
 'abdelatif',
 'abdelhakim',
 'abdelilah',
 'abdelkad',
 'abdic',
 'abdomin',
 'abdou',
 'abdu',
 'abduct',
 'abducte',
 'ab

### cosine similarity matrix

In [45]:
%%time
movie_desc_sim = linear_kernel(tfidf_matrix)#, tfidf_matrix)

CPU times: user 22.1 s, sys: 20.5 s, total: 42.6 s
Wall time: 1min


In [46]:
movie_desc_sim.shape

(45369, 45369)

In [ ]:
np.savez_compressed("COSINE_SIM.npz", movie_desc_sim)

### index mapping

In [70]:
# movie_id = movie_meta.id
TITLE_TO_INDEX = pd.Series(movie_meta.index, index=movie_meta.title)
ID_TO_INDEX = pd.Series(movie_meta.index, index=movie_meta.id)
INDEX_TO_ID = pd.Series(movie_meta.id, index=movie_meta.index)
INDEX_TO_TITLE = pd.Series(movie_meta.title, index=movie_meta.index)
ID_TO_INDEX = pd.Series(movie_meta.index, index=movie_meta.id)

In [ ]:
TITLE_TO_INDEX

In [ ]:
TITLE_TO_INDEX["Jumanji"]

In [74]:
ID_TO_INDEX

id
862           0
8844          1
15602         2
31357         3
11862         4
          ...  
30840     45364
111109    45365
67758     45366
227506    45367
461257    45368
Length: 45369, dtype: int64

In [75]:
INDEX_TO_ID[2999]

'863'

In [66]:
TITLE_TO_INDEX["Toy Story"]

0

In [81]:
movie_meta[movie_meta.index==0].values

array([['862', 'Toy Story',
        list(['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney', 'Wallace Shawn', 'John Ratzenberger', 'Annie Potts', 'John Morris', 'Laurie Metcalf', 'Sarah Freeman', 'Penn Jillette']),
        list(['jealousy', 'toy', 'boy', 'friendship', 'friends', 'rivalry', 'boy next door', 'new toy', 'toy comes to life']),
        list(['Animation', 'Comedy', 'Family']), 7.7, 5415,
        "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.",
        23950, '21.946943', '', list(['John Lasseter']),
        "Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate 

## movie description cosine similarity model

In [49]:
def get_recommendations(movie_id, cosine_sim, title_series):
    idx = ID_TO_INDEX[str(movie_id)]
    print(f"Recommend for `{INDEX_TO_TITLE[idx]}`")
    # TODO: duplicate name
#     if len(idx) > 1:
#         print(type(idx))
#         idx = idx.iloc[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return title_series.iloc[movie_indices]

In [78]:
get_recommendations(INDEX_TO_ID[10311], movie_desc_sim, movie_meta.title)

Recommend for `The 40 Year Old Virgin`


23823                     Andy Hardy's Blonde Trouble
3059                                  Man on the Moon
43347                Andy Kaufman Plays Carnegie Hall
15357                                     Toy Story 3
38419    Superstar: The Life and Times of Andy Warhol
42645    Andy Peters: Exclamation Mark Question Point
2037                                   Pretty in Pink
8336                                        The Champ
40196                                   Wabash Avenue
6026                     How to Lose a Guy in 10 Days
31412                                       Expecting
0                                           Toy Story
27181                      Life Begins for Andy Hardy
11619                                    Factory Girl
29342                  Andy Hardy's Private Secretary
7624      The First $20 Million Is Always the Hardest
34545                                   Lost and Love
21347                        Andy Hardy's Double Life
41830                       

In [ ]:
TITLE_TO_INDEX["The Godfather"]

In [ ]:
movie_meta[movie_meta.title=="The Godfather"]

## one-hot similarity model

In [ ]:
def one_hot_encoding(list_, db):
    res = np.zeros(len(db), dtype=int)
    for item in list_:
        idx_list = np.where(db == item)[0]
        if len(idx_list) == 0: continue
        res[idx_list[0]] = 1
    return list(res)#.reshape(1, -1)

In [ ]:
# def one_hot_encoding(list_, db):
#     res = np.zeros_like(db)
#     for item in list_:
#         res[np.where(db == item)[0][0]] = 1
#     return res.reshape(1, -1)

In [ ]:
def get_count(pd_series, threshold):
    """
    Get item count from list of itmes of the column.
    
    Args:
        pd_series (pd.Series)
        threshold (int)
    
    Returns:
        (pd.Series)
        (list)
    """
    cnt = pd_series.explode().to_frame("item").reset_index(drop=True).groupby("item").size().sort_values(ascending=False)
    series = cnt[cnt >= threshold]
    return series, series.index.tolist()

In [ ]:
_, DIRECTORS = get_count(movie_meta.director, threshold=40)
DIRECTORS = np.array(DIRECTORS)#, dtype=str)
DIRECTORS

In [ ]:
_, KEYWORDS = get_count(movie_meta.keywords, threshold=500)
KEYWORDS = np.array(KEYWORDS)#, dtype=str)
KEYWORDS

In [ ]:
_, CASTS = get_count(movie_meta.cast, threshold=100)
CASTS = np.array(CASTS)#, dtype=str)
CASTS

In [ ]:
GENRES = movie_meta.genres.explode().unique()
GENRES

In [ ]:
one_hot_name = GENRES.tolist() + KEYWORDS.tolist() + CASTS.tolist() + DIRECTORS.tolist()
one_hot_name

In [ ]:
movie_meta["genres"].apply(one_hot_encoding, db=GENRES)

In [ ]:
movie_meta["keywords"].apply(one_hot_encoding, db=KEYWORDS)

In [ ]:
movie_meta["cast"].apply(one_hot_encoding, db=CASTS)

In [ ]:
movie_meta["director"].apply(one_hot_encoding, db=DIRECTORS)

In [ ]:
movie_meta["one_hot"] = movie_meta["genres"].apply(one_hot_encoding, db=GENRES) + \
                        movie_meta["keywords"].apply(one_hot_encoding, db=KEYWORDS) + \
                        movie_meta["cast"].apply(one_hot_encoding, db=CASTS) + \
                        movie_meta["director"].apply(one_hot_encoding, db=DIRECTORS)

In [ ]:
len(movie_meta.one_hot[0])

### Jaccard Similarity

In [ ]:
# from scipy.spatial.distance import jaccard, pdist, squareform
# from sklearn.metrics import pairwise_distances
# from sklearn.metrics import jaccard_score
# from nltk.metrics.distance import jaccard_distance

In [ ]:
# %%time
# JACCARD_SIM = 1 - pdist(pd.DataFrame(movie_meta['one_hot'].explode().values.tolist()), "jaccard")
# JACCARD_SIM

In [ ]:
# %%time
# JACCARD_SIM_MAT = pd.DataFrame(squareform(JACCARD_SIM), index=movie_meta.index, columns= movie_meta.index)

In [ ]:
def Jaccard(s1, s2):
    assert isinstance(s1, set)
    assert isinstance(s2, set)
    nom = len(s1.intersection(s2))
    den = len(s1.union(s2))
    return nom / den

In [ ]:
movie_meta["items_for_jaccard"] = movie_meta["cast"] + movie_meta["keywords"] + movie_meta["genres"] + movie_meta["director"]
movie_meta

In [ ]:
def remove_empty_list(x):
#     print(type(x))
    assert isinstance(x, list)
    if len(x) == 0:
        return None
    else:
        return x

In [ ]:
# movie_meta.items_for_jaccard.apply(len).sort_values().head(45369-45116+1)

In [ ]:
# movie_meta["items_for_jaccard"].dropna(inplace=True)
movie_meta["items_for_jaccard"] = movie_meta["items_for_jaccard"].apply(remove_empty_list).dropna()


In [ ]:
movie_meta_new = movie_meta.dropna().reset_index()
movie_meta_new

In [ ]:
np.save("jaccard_sim_idx_mapping", movie_meta_new["index"].values)

In [ ]:
%%time
JACCARD_SIM = np.zeros((len(movie_meta_new.index), len(movie_meta_new.index)))
JACCARD_SIM, JACCARD_SIM.shape

In [ ]:
%%time
t = tqdm()
t.clear()
for i1 in tqdm(range(len(movie_meta_new.index))):
    for i2 in range(i1 + 1, len(movie_meta_new.index)):
        s1 = set(movie_meta_new.items_for_jaccard[i1])
        s2 = set(movie_meta_new.items_for_jaccard[i2])
#         if not s1.difference(s2): continue
        try:
            sim = Jaccard(s1, s2)
        except:
            print(f"Jaccard has trouble: (s1, s2) = ({s1}, {s2})")
            sim = 0
        JACCARD_SIM[i1, i2] = sim
        JACCARD_SIM[i2, i1] = sim

In [ ]:
np.savez_compressed("JACCARD_SIM.npz", JACCARD_SIM)

In [ ]:
a = np.load("JACCARD_SIM.npz")

In [ ]:
a.files

In [ ]:
%%time
aa = a["arr_0"]

In [ ]:
np.array(((1,1), (1,2))) + np.array(((1,1), (1,2)))